# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "baig_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [32]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

588

In [33]:
cities_sub = cities[0:10]
cities_sub

['bengkulu',
 'eyl',
 'sentyabrskiy',
 'ponta do sol',
 'zavetnoye',
 'port elizabeth',
 'avarua',
 'bluff',
 'surdulica',
 'ust-kamchatsk']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [34]:
#init lists to hold parsed data
lats = []
longs = [] 
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = clouds

df.head(10)

Got data for city index: 0 of 588
Missing data in OpenWeatherAPI for sentyabrskiy
Got data for city index: 5 of 588
Missing data in OpenWeatherAPI for ust-kamchatsk
Got data for city index: 10 of 588
Got data for city index: 15 of 588
Got data for city index: 20 of 588
Got data for city index: 25 of 588
Got data for city index: 30 of 588
Missing data in OpenWeatherAPI for illoqqortoormiut
Got data for city index: 35 of 588
Got data for city index: 40 of 588
Got data for city index: 45 of 588
Missing data in OpenWeatherAPI for attawapiskat
Got data for city index: 50 of 588
Got data for city index: 55 of 588
Got data for city index: 60 of 588
Missing data in OpenWeatherAPI for halalo
Missing data in OpenWeatherAPI for taolanaro
Got data for city index: 65 of 588
Got data for city index: 70 of 588
Got data for city index: 75 of 588
Got data for city index: 80 of 588
Got data for city index: 85 of 588
Missing data in OpenWeatherAPI for satitoa
Got data for city index: 90 of 588
Got data f

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Bengkulu,-3.80,102.27,75.42,80.76,84,1011,4.27,63
1,Eyl,7.98,49.82,76.32,75.34,66,1013,10.42,63
2,Ponta do Sol,32.67,-17.10,59.02,53.26,59,1014,8.05,63
3,Zavetnoye,47.12,43.89,46.42,34.70,69,1014,15.82,63
4,Port Elizabeth,-33.92,25.57,57.20,54.97,87,1011,5.82,63
5,Avarua,-21.21,-159.78,77.00,79.90,78,1013,8.05,63
6,Bluff,-46.60,168.33,51.44,45.00,78,1004,9.28,63
7,Surdulica,42.69,22.17,48.52,40.03,10,988,3.53,63
8,Hobart,-42.88,147.33,50.90,42.53,53,1022,9.17,63
9,San Ramon,37.78,-121.98,59.16,46.40,47,1008,18.34,63


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [44]:
 df.to_csv(output_data_file, index=False)